In [ ]:
!pip install langfuse openai aiohttp nest_asyncio pandas dotenv openpyxl python-dotenv

  Obtaining dependency information for openpyxl from https://files.pythonhosted.org/packages/c0/da/977ded879c29cbd04de313843e76868e6e13408a94ed6b987245dc7c8506/openpyxl-3.1.5-py2.py3-none-any.whl.metadata
  Obtaining dependency information for et-xmlfile from https://files.pythonhosted.org/packages/c1/8b/5fe2cc11fee489817272089c4203e679c63b570a5aaeb18d852ae3cbba6a/et_xmlfile-2.0.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 12.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Scenario 1: Order Delivery

## ✅ (V0) 초기 상태: 정보 없이 GPT 활용하기

In [ ]:
"""
Batch-processing (V0) for multiple GPT models **with latency & $ cost logging**
───────────────────────────────────────────────────────────────────────────────
• Reads environment variables from .env  
• Loads CSVs under data/01_order_delivery/  
• Generates **minimal** System="" & User="question" prompts (V0)  
• Runs 4 models (gpt‑4o‑mini, gpt‑4o, gpt‑4.1‑mini, gpt‑4.1) concurrently per row  
• Captures latency (ms) + token usage → USD cost (configurable)  
• Saves one Excel file per model:  
  Scenario_QA_with_responses_V0_<model>_<YYYYMMDD_HHMMSS>.xlsx
"""

import os
import asyncio
import time
from datetime import datetime
from typing import Dict, Any

import pandas as pd
from dotenv import load_dotenv
from langfuse.openai import AsyncOpenAI

# ──────────────────────────────────────────────────────────────────────────────
# Pricing table ($ per token)
# ──────────────────────────────────────────────────────────────────────────────
# NOTE: values = dollars / token (1M → divide by 1_000_000).
PRICING: Dict[str, Dict[str, float]] = {
    "gpt-4o-mini":   {"input": 0.15 / 1_000_000, "output": 0.60 / 1_000_000},
    "gpt-4o":        {"input": 2.50 / 1_000_000, "output": 20.00 / 1_000_000},
    "gpt-4.1-mini":  {"input": 0.40 / 1_000_000, "output": 1.60 / 1_000_000},
    "gpt-4.1":       {"input": 2.00 / 1_000_000, "output": 8.00  / 1_000_000},
}

# ──────────────────────────────────────────────────────────────────────────────
# 0. Notebook / Interactive safety
# ──────────────────────────────────────────────────────────────────────────────
try:
    import nest_asyncio  # type: ignore

    nest_asyncio.apply()
except ImportError:
    # Fine in plain .py execution
    pass

# ──────────────────────────────────────────────────────────────────────────────
# 1. Environment & client setup
# ──────────────────────────────────────────────────────────────────────────────

load_dotenv()  # Reads .env in project root

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")  # or use Azure_* if needed

if OPENAI_API_KEY is None:
    raise EnvironmentError("OPENAI_API_KEY not found in .env or env vars.")

client = AsyncOpenAI(api_key=OPENAI_API_KEY)

# ──────────────────────────────────────────────────────────────────────────────
# 2. Load CSVs & relational join
# ──────────────────────────────────────────────────────────────────────────────

BASE_DIR = os.path.join("../data", "01_order_delivery")

cust_df = pd.read_csv(os.path.join(BASE_DIR, "Customer_Info.csv"))
addr_df = pd.read_csv(os.path.join(BASE_DIR, "Delivery_Address.csv"))
order_df = pd.read_csv(os.path.join(BASE_DIR, "Order_Info.csv"))
issue_df = pd.read_csv(os.path.join(BASE_DIR, "Shipping_Issue_Log.csv"))  # Optional
scenario_df = pd.read_csv(os.path.join(BASE_DIR, "Scenario_QA.csv"))

# Merge to obtain the minimal context per scenario row
merged = (
    scenario_df
    .merge(cust_df, on="customer_id", how="left", suffixes=("", "_customer"))
    .merge(order_df, on="customer_id", how="left", suffixes=("", "_order"))
    .merge(addr_df, on="customer_id", how="left", suffixes=("", "_address"))
    .merge(issue_df, on="order_id", how="left", suffixes=("", "_issue"))
)

# ──────────────────────────────────────────────────────────────────────────────
# 3. Prompt builders
# ──────────────────────────────────────────────────────────────────────────────

def build_system_prompt(_: pd.Series) -> str:
    return ""  # V0 ⇒ no guidance

def build_user_prompt(row: pd.Series) -> str:
    return row.question

# ──────────────────────────────────────────────────────────────────────────────
# 4. Async call helper
# ──────────────────────────────────────────────────────────────────────────────

async def call_openai(row: pd.Series, model: str) -> Dict[str, Any]:
    start_ns = time.perf_counter_ns()

    resp = await client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": build_system_prompt(row)},
            {"role": "user",   "content": build_user_prompt(row)},
        ],
    )

    latency_ms = (time.perf_counter_ns() - start_ns) / 1_000_000  # → ms

    usage = resp.usage  # expect: prompt_tokens, completion_tokens
    prompt_toks = usage.prompt_tokens if usage else None
    completion_toks = usage.completion_tokens if usage else None

    price = PRICING[model]
    cost = None
    if prompt_toks is not None and completion_toks is not None:
        cost = prompt_toks * price["input"] + completion_toks * price["output"]

    answer = resp.choices[0].message.content.strip()
    return {
        "answer": answer,
        "latency_ms": latency_ms,
        "prompt_tokens": prompt_toks,
        "completion_tokens": completion_toks,
        "usd_cost": cost,
    }

# ──────────────────────────────────────────────────────────────────────────────
# 5. Runner per model
# ──────────────────────────────────────────────────────────────────────────────

async def run_model(model: str) -> None:
    print(f"Running model: {model} …")

    tasks = [call_openai(row, model) for _, row in merged.iterrows()]
    results = await asyncio.gather(*tasks)

    result_df = merged.copy()
    result_df[[
        f"answer_{model}",
        f"latency_ms_{model}",
        f"prompt_tok_{model}",
        f"completion_tok_{model}",
        f"usd_cost_{model}",
    ]] = pd.DataFrame([
        (r["answer"], r["latency_ms"], r["prompt_tokens"], r["completion_tokens"], r["usd_cost"]) for r in results
    ])

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    out_path = os.path.join(
        BASE_DIR,
        "answer_results",
        f"Scenario_QA_with_responses_V0_{model.replace('.', '_')}_{timestamp}.xlsx",
    )
    result_df.to_excel(out_path, index=False)
    print(f"✅ Saved → {out_path}")

# ──────────────────────────────────────────────────────────────────────────────
# 6. Main orchestrator: iterate over models
# ──────────────────────────────────────────────────────────────────────────────

MODELS = [
    "gpt-4o-mini",
    "gpt-4o",
    "gpt-4.1-mini",
    "gpt-4.1",
]

async def main() -> None:
    for model in MODELS:
        await run_model(model)

try:
    await main()
except RuntimeError:
    # Already inside event loop (e.g., Jupyter)
    import nest_asyncio  # re-apply in case user removed earlier
    nest_asyncio.apply()
    import asyncio as _aio
    _aio.get_event_loop().create_task(main())


Running model: gpt-4o-mini …
✅ Saved → ../data/01_order_delivery/answer_results/Scenario_QA_with_responses_V0_gpt-4o-mini_20250614_063249.xlsx
Running model: gpt-4o …
✅ Saved → ../data/01_order_delivery/answer_results/Scenario_QA_with_responses_V0_gpt-4o_20250614_063256.xlsx
Running model: gpt-4.1-mini …
✅ Saved → ../data/01_order_delivery/answer_results/Scenario_QA_with_responses_V0_gpt-4_1-mini_20250614_063259.xlsx
Running model: gpt-4.1 …
✅ Saved → ../data/01_order_delivery/answer_results/Scenario_QA_with_responses_V0_gpt-4_1_20250614_063310.xlsx


## 각 고객의 주문 정보, 주소 정보, 배송 이슈 정보들을 조회하여서 Question에 대한 답변을 할 때

In [ ]:
"""
Batch‑processing (Simple) script for multiple GPT models **with latency & $ cost logging**
───────────────────────────────────────────────────────────────────────────────
• Reads environment variables from .env  
• Loads CSVs under data/01_order_delivery/  
• Generates **minimal** System="" & User="question" prompts (V0)  
• Runs 4 models (gpt‑4o‑mini, gpt‑4o, gpt‑4.1‑mini, gpt‑4.1) concurrently per row  
• Captures latency (ms) + token usage → USD cost (configurable)  
• Saves one Excel file per model:  
  Scenario_QA_with_responses_Simple_<model>_<YYYYMMDD_HHMMSS>.xlsx
"""

import os
import asyncio
import time
from datetime import datetime
from typing import Dict, Any

import pandas as pd
from dotenv import load_dotenv
from langfuse import Langfuse
from langfuse.openai import AsyncOpenAI

# ──────────────────────────────────────────────────────────────────────────────
# Pricing table ($ per token)
# ──────────────────────────────────────────────────────────────────────────────
# NOTE: values = dollars / token (1M → divide by 1_000_000).
PRICING: Dict[str, Dict[str, float]] = {
    "gpt-4o-mini":   {"input": 0.15 / 1_000_000, "output": 0.60 / 1_000_000},
    "gpt-4o":        {"input": 2.50 / 1_000_000, "output": 20.00 / 1_000_000},
    "gpt-4.1-mini":  {"input": 0.40 / 1_000_000, "output": 1.60 / 1_000_000},
    "gpt-4.1":       {"input": 2.00 / 1_000_000, "output": 8.00  / 1_000_000},
}

# ──────────────────────────────────────────────────────────────────────────────
# 0. Notebook / Interactive safety
# ──────────────────────────────────────────────────────────────────────────────
try:
    import nest_asyncio  # type: ignore

    nest_asyncio.apply()
except ImportError:
    # Fine in plain .py execution
    pass

# ──────────────────────────────────────────────────────────────────────────────
# 1. Environment & client setup
# ──────────────────────────────────────────────────────────────────────────────

load_dotenv()  # Reads .env in project root

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")  # or use Azure_* if needed

if OPENAI_API_KEY is None:
    raise EnvironmentError("OPENAI_API_KEY not found in .env or env vars.")

client = AsyncOpenAI(api_key=OPENAI_API_KEY)
langfuse = Langfuse()
# ──────────────────────────────────────────────────────────────────────────────
# 2. Load CSVs & relational join
# ──────────────────────────────────────────────────────────────────────────────

BASE_DIR = os.path.join("../data", "01_order_delivery")

cust_df = pd.read_csv(os.path.join(BASE_DIR, "Customer_Info.csv"))
addr_df = pd.read_csv(os.path.join(BASE_DIR, "Delivery_Address.csv"))
order_df = pd.read_csv(os.path.join(BASE_DIR, "Order_Info.csv"))
issue_df = pd.read_csv(os.path.join(BASE_DIR, "Shipping_Issue_Log.csv"))  # Optional
scenario_df = pd.read_csv(os.path.join(BASE_DIR, "Scenario_QA.csv"))

# Merge to obtain the minimal context per scenario row
merged = (
    scenario_df
    .merge(cust_df, on="customer_id", how="left", suffixes=("", "_customer"))
    .merge(order_df, on="customer_id", how="left", suffixes=("", "_order"))
    .merge(addr_df, on="customer_id", how="left", suffixes=("", "_address"))
    .merge(issue_df, on="order_id", how="left", suffixes=("", "_issue"))
)

# ──────────────────────────────────────────────────────────────────────────────
# 3. Prompt builders
# ──────────────────────────────────────────────────────────────────────────────

def build_system_prompt(row: pd.Series) -> str:
    """Return a compact system prompt summarising customer/order key facts."""
    pieces = [
        f"Customer: {row.customer_name} (ID {row.customer_id})",
        f"Order: {row.product_name} (Order‑ID {row.order_id})",
        f"Shipping status: {row.shipping_status}",
    ]
    if pd.notna(row.shipping_company):
        pieces.append(f"Courier: {row.shipping_company} — Tracking: {row.tracking_number or 'pending'}")
    pieces.append(
        f"Primary address: {row.address_line1}, {row.city} {row.postal_code}"
    )
    return "\n".join(pieces)


def build_user_prompt(row: pd.Series) -> str:
    return row.question

# ──────────────────────────────────────────────────────────────────────────────
# 4. Async call helper
# ──────────────────────────────────────────────────────────────────────────────

async def call_openai(row: pd.Series, model: str) -> Dict[str, Any]:
    start_ns = time.perf_counter_ns()

    resp = await client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": build_system_prompt(row)},
            {"role": "user",   "content": build_user_prompt(row)},
        ],
    )

    latency_ms = (time.perf_counter_ns() - start_ns) / 1_000_000  # → ms

    usage = resp.usage  # expect: prompt_tokens, completion_tokens
    prompt_toks = usage.prompt_tokens if usage else None
    completion_toks = usage.completion_tokens if usage else None

    price = PRICING[model]
    cost = None
    if prompt_toks is not None and completion_toks is not None:
        cost = prompt_toks * price["input"] + completion_toks * price["output"]

    answer = resp.choices[0].message.content.strip()
    return {
        "answer": answer,
        "latency_ms": latency_ms,
        "prompt_tokens": prompt_toks,
        "completion_tokens": completion_toks,
        "usd_cost": cost,
    }

# ──────────────────────────────────────────────────────────────────────────────
# 5. Runner per model
# ──────────────────────────────────────────────────────────────────────────────

async def run_model(model: str) -> None:
    print(f"Running model: {model} …")

    tasks = [call_openai(row, model) for _, row in merged.iterrows()]
    results = await asyncio.gather(*tasks)

    result_df = merged.copy()
    result_df[[
        f"answer_{model}",
        f"latency_ms_{model}",
        f"prompt_tok_{model}",
        f"completion_tok_{model}",
        f"usd_cost_{model}",
    ]] = pd.DataFrame([
        (r["answer"], r["latency_ms"], r["prompt_tokens"], r["completion_tokens"], r["usd_cost"]) for r in results
    ])

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    out_path = os.path.join(
        BASE_DIR,
        "answer_results",
        f"Scenario_QA_with_responses_Simple_{model.replace('.', '_')}_{timestamp}.xlsx",
    )
    result_df.to_excel(out_path, index=False)
    print(f"✅ Saved → {out_path}")

# ──────────────────────────────────────────────────────────────────────────────
# 6. Main orchestrator: iterate over models
# ──────────────────────────────────────────────────────────────────────────────

MODELS = [
    "gpt-4o-mini",
    "gpt-4o",
    "gpt-4.1-mini",
    "gpt-4.1",
]

async def main() -> None:
    for model in MODELS:
        await run_model(model)

try:
    await main()
except RuntimeError:
    # Already inside event loop (e.g., Jupyter)
    import nest_asyncio  # re-apply in case user removed earlier
    nest_asyncio.apply()
    import asyncio as _aio
    _aio.get_event_loop().create_task(main())


Running model: gpt-4o-mini …
✅ Saved → ../data/01_order_delivery/answer_results/Scenario_QA_with_responses_Simple_gpt-4o-mini_20250614_064353.xlsx
Running model: gpt-4o …
✅ Saved → ../data/01_order_delivery/answer_results/Scenario_QA_with_responses_Simple_gpt-4o_20250614_064400.xlsx
Running model: gpt-4.1-mini …
✅ Saved → ../data/01_order_delivery/answer_results/Scenario_QA_with_responses_Simple_gpt-4_1-mini_20250614_064407.xlsx
Running model: gpt-4.1 …
✅ Saved → ../data/01_order_delivery/answer_results/Scenario_QA_with_responses_Simple_gpt-4_1_20250614_064417.xlsx


---
---
# ✅ Scenario 2: Refund

## ✅ (V0) 초기 상태: 정보 없이 GPT 활용하기

In [ ]:
"""
Batch-processing (V0) for multiple GPT models **with latency & $ cost logging**
───────────────────────────────────────────────────────────────────────────────
• Reads environment variables from .env  
• Loads CSVs under data/02_refund
• Generates **minimal** System="" & User="question" prompts (V0)  
• Runs 4 models (gpt‑4o‑mini, gpt‑4o, gpt‑4.1‑mini, gpt‑4.1) concurrently per row  
• Captures latency (ms) + token usage → USD cost (configurable)  
• Saves one Excel file per model:  
  Scenario_QA_with_responses_V0_<model>_<YYYYMMDD_HHMMSS>.xlsx
"""

import os
import asyncio
import time
from datetime import datetime
from typing import Dict, Any

import pandas as pd
from dotenv import load_dotenv
from langfuse import Langfuse
from langfuse.openai import AsyncOpenAI

# ──────────────────────────────────────────────────────────────────────────────
# Pricing table ($ per token)
# ──────────────────────────────────────────────────────────────────────────────
# NOTE: values = dollars / token (1M → divide by 1_000_000).
PRICING: Dict[str, Dict[str, float]] = {
    "gpt-4o-mini":   {"input": 0.15 / 1_000_000, "output": 0.60 / 1_000_000},
    "gpt-4o":        {"input": 2.50 / 1_000_000, "output": 20.00 / 1_000_000},
    "gpt-4.1-mini":  {"input": 0.40 / 1_000_000, "output": 1.60 / 1_000_000},
    "gpt-4.1":       {"input": 2.00 / 1_000_000, "output": 8.00  / 1_000_000},
}

# ─────────────────────────────────────────────────────
# 0. Async Safety
# ─────────────────────────────────────────────────────
try:
    import nest_asyncio
    nest_asyncio.apply()
except ImportError:
    pass

# ──────────────────────────────────────────────────────────────────────────────
# 1. Environment & client setup
# ──────────────────────────────────────────────────────────────────────────────

load_dotenv()  # Reads .env in project root

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")  # or use Azure_* if needed

if OPENAI_API_KEY is None:
    raise EnvironmentError("OPENAI_API_KEY not found in .env or env vars.")

client = AsyncOpenAI(api_key=OPENAI_API_KEY)
langfuse = Langfuse()

# ─────────────────────────────────────────────────────
# 2. Load Refund Scenario CSVs
# ─────────────────────────────────────────────────────
BASE_DIR = os.path.join("..", "data", "02_refund")

cust_df = pd.read_csv(os.path.join(BASE_DIR, "T_CUSTOMER.csv"))
order_df = pd.read_csv(os.path.join(BASE_DIR, "T_ORDER.csv"))
pay_df = pd.read_csv(os.path.join(BASE_DIR, "T_PAYMENT.csv"))
refund_df = pd.read_csv(os.path.join(BASE_DIR, "T_REFUND.csv"))
scenario_df = pd.read_csv(os.path.join(BASE_DIR, "Scenario_QA.csv"))

# Merge all necessary tables for contextual prompt building
merged = (
    scenario_df
    .merge(cust_df, left_on="CustomerName", right_on="Name", how="left")
    .merge(order_df, on="CustomerID", how="left")
    .merge(pay_df, on="OrderID", how="left")
    .merge(refund_df, on="PaymentID", how="left", suffixes=("", "_refund"))
)

# ─────────────────────────────────────────────────────
# 3. Prompt Construction Functions
# ─────────────────────────────────────────────────────
def build_system_prompt(row: pd.Series) -> str:
    # V0: no context at all
    return ""        # or: return "You are a helpful assistant."


def build_user_prompt(row: pd.Series) -> str:
    return row["Question"]

# ──────────────────────────────────────────────────────────────────────────────
# 4. Async call helper
# ──────────────────────────────────────────────────────────────────────────────

async def call_openai(row: pd.Series, model: str) -> Dict[str, Any]:
    start_ns = time.perf_counter_ns()

    resp = await client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": build_system_prompt(row)},
            {"role": "user",   "content": build_user_prompt(row)},
        ],
    )

    latency_ms = (time.perf_counter_ns() - start_ns) / 1_000_000  # → ms

    usage = resp.usage  # expect: prompt_tokens, completion_tokens
    prompt_toks = usage.prompt_tokens if usage else None
    completion_toks = usage.completion_tokens if usage else None

    price = PRICING[model]
    cost = None
    if prompt_toks is not None and completion_toks is not None:
        cost = prompt_toks * price["input"] + completion_toks * price["output"]

    answer = resp.choices[0].message.content.strip()
    return {
        "answer": answer,
        "latency_ms": latency_ms,
        "prompt_tokens": prompt_toks,
        "completion_tokens": completion_toks,
        "usd_cost": cost,
    }

# ──────────────────────────────────────────────────────────────────────────────
# 5. Runner per model
# ──────────────────────────────────────────────────────────────────────────────

async def run_model(model: str) -> None:
    print(f"Running model: {model} …")

    tasks = [call_openai(row, model) for _, row in merged.iterrows()]
    results = await asyncio.gather(*tasks)

    result_df = merged.copy()
    result_df[[
        f"answer_{model}",
        f"latency_ms_{model}",
        f"prompt_tok_{model}",
        f"completion_tok_{model}",
        f"usd_cost_{model}",
    ]] = pd.DataFrame([
        (r["answer"], r["latency_ms"], r["prompt_tokens"], r["completion_tokens"], r["usd_cost"]) for r in results
    ])

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    out_path = os.path.join(
        BASE_DIR,
        "answer_results",
        f"Scenario_QA_with_responses_V0_{model.replace('.', '_')}_{timestamp}.xlsx",
    )
    result_df.to_excel(out_path, index=False)
    print(f"✅ Saved → {out_path}")

# ─────────────────────────────────────────────────────
# 6. Orchestrator for Multiple Models
# ─────────────────────────────────────────────────────
MODELS = [
    "gpt-4o-mini",
    "gpt-4o",
    "gpt-4.1-mini",
    "gpt-4.1",
]

async def main() -> None:
    for model in MODELS:
        await run_model(model)


try:
    await main()
except RuntimeError:
    import nest_asyncio
    nest_asyncio.apply()
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())


Running model: gpt-4o-mini …
✅ Saved → ../data/02_refund/answer_results/Scenario_QA_with_responses_V0_gpt-4o-mini_20250614_064423.xlsx
Running model: gpt-4o …
✅ Saved → ../data/02_refund/answer_results/Scenario_QA_with_responses_V0_gpt-4o_20250614_064431.xlsx
Running model: gpt-4.1-mini …
✅ Saved → ../data/02_refund/answer_results/Scenario_QA_with_responses_V0_gpt-4_1-mini_20250614_064437.xlsx
Running model: gpt-4.1 …
✅ Saved → ../data/02_refund/answer_results/Scenario_QA_with_responses_V0_gpt-4_1_20250614_064458.xlsx


## 각 고객의 고객 정보, 주문 정보, 결제 정보, 환불 정보를 조회하여서 Question에 대한 답변을 할 때

In [ ]:
"""
Batch-processing (Simple) for multiple GPT models **with latency & $ cost logging**
───────────────────────────────────────────────────────────────────────────────
• Reads environment variables from .env  
• Loads CSVs under data/02_refund
• Generates **minimal** System="" & User="question" prompts (V0)  
• Runs 4 models (gpt‑4o‑mini, gpt‑4o, gpt‑4.1‑mini, gpt‑4.1) concurrently per row  
• Captures latency (ms) + token usage → USD cost (configurable)  
• Saves one Excel file per model:  
  Scenario_QA_with_responses_Simple_<model>_<YYYYMMDD_HHMMSS>.xlsx
"""

import os
import asyncio
import time
from datetime import datetime
from typing import Dict, Any

import pandas as pd
from dotenv import load_dotenv
from langfuse import Langfuse
from langfuse.openai import AsyncOpenAI

# ──────────────────────────────────────────────────────────────────────────────
# Pricing table ($ per token)
# ──────────────────────────────────────────────────────────────────────────────
# NOTE: values = dollars / token (1M → divide by 1_000_000).
PRICING: Dict[str, Dict[str, float]] = {
    "gpt-4o-mini":   {"input": 0.15 / 1_000_000, "output": 0.60 / 1_000_000},
    "gpt-4o":        {"input": 2.50 / 1_000_000, "output": 20.00 / 1_000_000},
    "gpt-4.1-mini":  {"input": 0.40 / 1_000_000, "output": 1.60 / 1_000_000},
    "gpt-4.1":       {"input": 2.00 / 1_000_000, "output": 8.00  / 1_000_000},
}

# ─────────────────────────────────────────────────────
# 0. Async Safety
# ─────────────────────────────────────────────────────
try:
    import nest_asyncio
    nest_asyncio.apply()
except ImportError:
    pass

# ──────────────────────────────────────────────────────────────────────────────
# 1. Environment & client setup
# ──────────────────────────────────────────────────────────────────────────────

load_dotenv()  # Reads .env in project root

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")  # or use Azure_* if needed

if OPENAI_API_KEY is None:
    raise EnvironmentError("OPENAI_API_KEY not found in .env or env vars.")

client = AsyncOpenAI(api_key=OPENAI_API_KEY)
langfuse = Langfuse()

# ─────────────────────────────────────────────────────
# 2. Load Refund Scenario CSVs
# ─────────────────────────────────────────────────────
BASE_DIR = os.path.join("..", "data", "02_refund")

cust_df = pd.read_csv(os.path.join(BASE_DIR, "T_CUSTOMER.csv"))
order_df = pd.read_csv(os.path.join(BASE_DIR, "T_ORDER.csv"))
pay_df = pd.read_csv(os.path.join(BASE_DIR, "T_PAYMENT.csv"))
refund_df = pd.read_csv(os.path.join(BASE_DIR, "T_REFUND.csv"))
scenario_df = pd.read_csv(os.path.join(BASE_DIR, "Scenario_QA.csv"))

# Merge all necessary tables for contextual prompt building
merged = (
    scenario_df
    .merge(cust_df, left_on="CustomerName", right_on="Name", how="left")
    .merge(order_df, on="CustomerID", how="left")
    .merge(pay_df, on="OrderID", how="left")
    .merge(refund_df, on="PaymentID", how="left", suffixes=("", "_refund"))
)

# ─────────────────────────────────────────────────────
# 3. Prompt Construction Functions
# ─────────────────────────────────────────────────────
def build_system_prompt(row: pd.Series) -> str:
    lines = [
        f"Customer: {row['Name']} (ID {row['CustomerID']})",
        f"Order: {row['ProductName']} ×{row['Quantity']} (OrderID: {row['OrderID']})",
        f"Payment: ₩{row['PaymentAmount']} via {row['PaymentMethod']} on {row['PaymentDate']} (Status: {row['PaymentStatus']})",
    ]
    if pd.notna(row.get("RefundStatus")):
        lines.append(
            f"Refund: Requested on {row['RefundRequestDate']} (Status: {row['RefundStatus']})"
        )
    return "\n".join(lines)


def build_user_prompt(row: pd.Series) -> str:
    return row["Question"]

# ──────────────────────────────────────────────────────────────────────────────
# 4. Async call helper
# ──────────────────────────────────────────────────────────────────────────────

async def call_openai(row: pd.Series, model: str) -> Dict[str, Any]:
    start_ns = time.perf_counter_ns()

    resp = await client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": build_system_prompt(row)},
            {"role": "user",   "content": build_user_prompt(row)},
        ],
    )

    latency_ms = (time.perf_counter_ns() - start_ns) / 1_000_000  # → ms

    usage = resp.usage  # expect: prompt_tokens, completion_tokens
    prompt_toks = usage.prompt_tokens if usage else None
    completion_toks = usage.completion_tokens if usage else None

    price = PRICING[model]
    cost = None
    if prompt_toks is not None and completion_toks is not None:
        cost = prompt_toks * price["input"] + completion_toks * price["output"]

    answer = resp.choices[0].message.content.strip()
    return {
        "answer": answer,
        "latency_ms": latency_ms,
        "prompt_tokens": prompt_toks,
        "completion_tokens": completion_toks,
        "usd_cost": cost,
    }

# ──────────────────────────────────────────────────────────────────────────────
# 5. Runner per model
# ──────────────────────────────────────────────────────────────────────────────

async def run_model(model: str) -> None:
    print(f"Running model: {model} …")

    tasks = [call_openai(row, model) for _, row in merged.iterrows()]
    results = await asyncio.gather(*tasks)

    result_df = merged.copy()
    result_df[[
        f"answer_{model}",
        f"latency_ms_{model}",
        f"prompt_tok_{model}",
        f"completion_tok_{model}",
        f"usd_cost_{model}",
    ]] = pd.DataFrame([
        (r["answer"], r["latency_ms"], r["prompt_tokens"], r["completion_tokens"], r["usd_cost"]) for r in results
    ])

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    out_path = os.path.join(
        BASE_DIR,
        "answer_results",
        f"Scenario_QA_with_responses_Simple_{model.replace('.', '_')}_{timestamp}.xlsx",
    )
    result_df.to_excel(out_path, index=False)
    print(f"✅ Saved → {out_path}")

# ─────────────────────────────────────────────────────
# 6. Orchestrator for Multiple Models
# ─────────────────────────────────────────────────────
MODELS = [
    "gpt-4o-mini",
    "gpt-4o",
    "gpt-4.1-mini",
    "gpt-4.1",
]

async def main() -> None:
    for model in MODELS:
        await run_model(model)


try:
    await main()
except RuntimeError:
    import nest_asyncio
    nest_asyncio.apply()
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())


Running model: gpt-4o-mini …
✅ Saved → ../data/02_refund/answer_results/Scenario_QA_with_responses_Simple_gpt-4o-mini_20250614_064504.xlsx
Running model: gpt-4o …
✅ Saved → ../data/02_refund/answer_results/Scenario_QA_with_responses_Simple_gpt-4o_20250614_064516.xlsx
Running model: gpt-4.1-mini …
✅ Saved → ../data/02_refund/answer_results/Scenario_QA_with_responses_Simple_gpt-4_1-mini_20250614_064519.xlsx
Running model: gpt-4.1 …
✅ Saved → ../data/02_refund/answer_results/Scenario_QA_with_responses_Simple_gpt-4_1_20250614_064602.xlsx


# ✅ Scenario 3: Account Login

## ✅ (V0) 초기 상태: 정보 없이 GPT 활용하기

In [ ]:
"""
Batch-processing (V0) for multiple GPT models **with latency & $ cost logging**
───────────────────────────────────────────────────────────────────────────────
• Reads environment variables from .env  
• Loads CSVs under data/03_account_login
• Generates **minimal** System="" & User="question" prompts (V0)  
• Runs 4 models (gpt‑4o‑mini, gpt‑4o, gpt‑4.1‑mini, gpt‑4.1) concurrently per row  
• Captures latency (ms) + token usage → USD cost (configurable)  
• Saves one Excel file per model:  
  Scenario_QA_with_responses_V0_<model>_<YYYYMMDD_HHMMSS>.xlsx
"""

import os
import asyncio
import time
from datetime import datetime
from typing import Dict, Any

import pandas as pd
from dotenv import load_dotenv
from langfuse import Langfuse
from langfuse.openai import AsyncOpenAI

# ──────────────────────────────────────────────────────────────────────────────
# Pricing table ($ per token)
# ──────────────────────────────────────────────────────────────────────────────
# NOTE: values = dollars / token (1M → divide by 1_000_000).
PRICING: Dict[str, Dict[str, float]] = {
    "gpt-4o-mini":   {"input": 0.15 / 1_000_000, "output": 0.60 / 1_000_000},
    "gpt-4o":        {"input": 2.50 / 1_000_000, "output": 20.00 / 1_000_000},
    "gpt-4.1-mini":  {"input": 0.40 / 1_000_000, "output": 1.60 / 1_000_000},
    "gpt-4.1":       {"input": 2.00 / 1_000_000, "output": 8.00  / 1_000_000},
}

# ─────────────────────────────────────────────────────
# 0. Async Safety
# ─────────────────────────────────────────────────────
try:
    import nest_asyncio
    nest_asyncio.apply()
except ImportError:
    pass

# ──────────────────────────────────────────────────────────────────────────────
# 1. 환경 설정 및 클라이언트 준비
# ──────────────────────────────────────────────────────────────────────────────

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise EnvironmentError("OPENAI_API_KEY not found in .env")

client = AsyncOpenAI(api_key=OPENAI_API_KEY)
langfuse = Langfuse()

# ──────────────────────────────────────────────────────────────────────────────
# 2. CSV 로드 및 조인
# ──────────────────────────────────────────────────────────────────────────────

BASE_DIR = "../data/03_account_login"
RESULT_DIR = os.path.join(BASE_DIR)

user_df = pd.read_csv(os.path.join(RESULT_DIR, "user_accounts.csv"))
login_df = pd.read_csv(os.path.join(RESULT_DIR, "login_history.csv"))
security_df = pd.read_csv(os.path.join(RESULT_DIR, "security_info.csv"))
ticket_df = pd.read_csv(os.path.join(RESULT_DIR, "support_tickets.csv"))
scenario_df = pd.read_csv(os.path.join(RESULT_DIR, "Scenario_QA_Cases_Cleaned.csv"))

# ticket_id가 있는 고객 기준으로 데이터 통합
merged = (
    scenario_df
    .merge(ticket_df, on="user_id", how="left")
    .merge(user_df, on="user_id", how="left")
    .merge(security_df, on="user_id", how="left")
    .merge(login_df.groupby("user_id").agg({"login_ip": "last", "login_timestamp": "last"}).reset_index(),
           on="user_id", how="left")
)

# ─────────────────────────────────────────────────────
# 3. Prompt 구성 함수
# ─────────────────────────────────────────────────────
def build_system_prompt(row: pd.Series) -> str:
    # V0: no context at all
    return ""        # or: return "You are a helpful assistant."

def build_user_prompt(row: pd.Series) -> str:
    return row["question"]

# ──────────────────────────────────────────────────────────────────────────────
# 4. Async call helper
# ──────────────────────────────────────────────────────────────────────────────

async def call_openai(row: pd.Series, model: str) -> Dict[str, Any]:
    start_ns = time.perf_counter_ns()

    resp = await client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": build_system_prompt(row)},
            {"role": "user",   "content": build_user_prompt(row)},
        ],
    )

    latency_ms = (time.perf_counter_ns() - start_ns) / 1_000_000  # → ms

    usage = resp.usage  # expect: prompt_tokens, completion_tokens
    prompt_toks = usage.prompt_tokens if usage else None
    completion_toks = usage.completion_tokens if usage else None

    price = PRICING[model]
    cost = None
    if prompt_toks is not None and completion_toks is not None:
        cost = prompt_toks * price["input"] + completion_toks * price["output"]

    answer = resp.choices[0].message.content.strip()
    return {
        "answer": answer,
        "latency_ms": latency_ms,
        "prompt_tokens": prompt_toks,
        "completion_tokens": completion_toks,
        "usd_cost": cost,
    }

# ──────────────────────────────────────────────────────────────────────────────
# 5. Runner per model
# ──────────────────────────────────────────────────────────────────────────────

async def run_model(model: str) -> None:
    print(f"Running model: {model} …")

    tasks = [call_openai(row, model) for _, row in merged.iterrows()]
    results = await asyncio.gather(*tasks)

    result_df = merged.copy()
    result_df[[
        f"answer_{model}",
        f"latency_ms_{model}",
        f"prompt_tok_{model}",
        f"completion_tok_{model}",
        f"usd_cost_{model}",
    ]] = pd.DataFrame([
        (r["answer"], r["latency_ms"], r["prompt_tokens"], r["completion_tokens"], r["usd_cost"]) for r in results
    ])

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    out_path = os.path.join(
        BASE_DIR,
        "answer_results",
        f"Scenario_QA_with_responses_V0_{model.replace('.', '_')}_{timestamp}.xlsx",
    )
    result_df.to_excel(out_path, index=False)
    print(f"✅ Saved → {out_path}")

# ──────────────────────────────────────────────────────────────────────────────
# 6. Main – 모델 리스트 실행
# ──────────────────────────────────────────────────────────────────────────────

MODELS = ["gpt-4.1-mini", "gpt-4.1", "gpt-4o", "gpt-4o-mini"]

async def main():
    for model in MODELS:
        await run_model(model)

# 실행 트리거 (스크립트 or 노트북 환경 모두 대응)
try:
    await main()
except RuntimeError:
    nest_asyncio.apply()
    asyncio.get_event_loop().create_task(main())


Running model: gpt-4.1-mini …
✅ Saved → ../data/03_account_login/answer_results/Scenario_QA_with_responses_V0_gpt-4_1-mini_20250614_064608.xlsx
Running model: gpt-4.1 …
✅ Saved → ../data/03_account_login/answer_results/Scenario_QA_with_responses_V0_gpt-4_1_20250614_064623.xlsx
Running model: gpt-4o …
✅ Saved → ../data/03_account_login/answer_results/Scenario_QA_with_responses_V0_gpt-4o_20250614_064642.xlsx
Running model: gpt-4o-mini …
✅ Saved → ../data/03_account_login/answer_results/Scenario_QA_with_responses_V0_gpt-4o-mini_20250614_064652.xlsx


In [8]:
## 각 고객의 고객 정보, 주문 정보, 결제 정보, 환불 정보를 조회하여서 Question에 대한 답변을 할 때

In [ ]:
"""
Batch-processing (Simple) for multiple GPT models **with latency & $ cost logging**
───────────────────────────────────────────────────────────────────────────────
• Reads environment variables from .env  
• Loads CSVs under data/03_account_login
• Generates **minimal** System="" & User="question" prompts (Simple)  
• Runs 4 models (gpt‑4o‑mini, gpt‑4o, gpt‑4.1‑mini, gpt‑4.1) concurrently per row  
• Captures latency (ms) + token usage → USD cost (configurable)  
• Saves one Excel file per model:  
  Scenario_QA_with_responses_Simple_<model>_<YYYYMMDD_HHMMSS>.xlsx
"""

import os
import asyncio
import time
from datetime import datetime
from typing import Dict, Any

import pandas as pd
from dotenv import load_dotenv
from langfuse import Langfuse
from langfuse.openai import AsyncOpenAI

# ──────────────────────────────────────────────────────────────────────────────
# Pricing table ($ per token)
# ──────────────────────────────────────────────────────────────────────────────
# NOTE: values = dollars / token (1M → divide by 1_000_000).
PRICING: Dict[str, Dict[str, float]] = {
    "gpt-4o-mini":   {"input": 0.15 / 1_000_000, "output": 0.60 / 1_000_000},
    "gpt-4o":        {"input": 2.50 / 1_000_000, "output": 20.00 / 1_000_000},
    "gpt-4.1-mini":  {"input": 0.40 / 1_000_000, "output": 1.60 / 1_000_000},
    "gpt-4.1":       {"input": 2.00 / 1_000_000, "output": 8.00  / 1_000_000},
}

# ─────────────────────────────────────────────────────
# 0. Async Safety
# ─────────────────────────────────────────────────────
try:
    import nest_asyncio
    nest_asyncio.apply()
except ImportError:
    pass
# ──────────────────────────────────────────────────────────────────────────────
# 1. 환경 설정 및 클라이언트 준비
# ──────────────────────────────────────────────────────────────────────────────

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise EnvironmentError("OPENAI_API_KEY not found in .env")

client = AsyncOpenAI(api_key=OPENAI_API_KEY)
langfuse = Langfuse()

# ──────────────────────────────────────────────────────────────────────────────
# 2. CSV 로드 및 조인
# ──────────────────────────────────────────────────────────────────────────────

BASE_DIR = "../data/03_account_login"
RESULT_DIR = os.path.join(BASE_DIR)

user_df = pd.read_csv(os.path.join(RESULT_DIR, "user_accounts.csv"))
login_df = pd.read_csv(os.path.join(RESULT_DIR, "login_history.csv"))
security_df = pd.read_csv(os.path.join(RESULT_DIR, "security_info.csv"))
ticket_df = pd.read_csv(os.path.join(RESULT_DIR, "support_tickets.csv"))
scenario_df = pd.read_csv(os.path.join(RESULT_DIR, "Scenario_QA_Cases_Cleaned.csv"))

# ticket_id가 있는 고객 기준으로 데이터 통합
merged = (
    scenario_df
    .merge(ticket_df, on="user_id", how="left")
    .merge(user_df, on="user_id", how="left")
    .merge(security_df, on="user_id", how="left")
    .merge(login_df.groupby("user_id").agg({"login_ip": "last", "login_timestamp": "last"}).reset_index(),
           on="user_id", how="left")
)

# ──────────────────────────────────────────────────────────────────────────────
# 3. Prompt 구성 함수
# ──────────────────────────────────────────────────────────────────────────────

def build_system_prompt(row: pd.Series) -> str:
    return "\n".join([
        f"User ID: {row.user_id}",
        f"Account status: {row.account_status}",
        f"Creation date: {row.creation_date or 'unknown'}",
        f"Last login IP: {row.login_ip or 'unknown'}",
        f"2FA Enabled: {row.two_factor_enabled} via {row.two_factor_method or 'N/A'}",
        f"Support ticket: {row.issue_description or 'no issue'}",
    ])

def build_user_prompt(row: pd.Series) -> str:
    return row["question"]

# ──────────────────────────────────────────────────────────────────────────────
# 4. Async call helper
# ──────────────────────────────────────────────────────────────────────────────

async def call_openai(row: pd.Series, model: str) -> Dict[str, Any]:
    start_ns = time.perf_counter_ns()

    resp = await client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": build_system_prompt(row)},
            {"role": "user",   "content": build_user_prompt(row)},
        ],
    )

    latency_ms = (time.perf_counter_ns() - start_ns) / 1_000_000  # → ms

    usage = resp.usage  # expect: prompt_tokens, completion_tokens
    prompt_toks = usage.prompt_tokens if usage else None
    completion_toks = usage.completion_tokens if usage else None

    price = PRICING[model]
    cost = None
    if prompt_toks is not None and completion_toks is not None:
        cost = prompt_toks * price["input"] + completion_toks * price["output"]

    answer = resp.choices[0].message.content.strip()
    return {
        "answer": answer,
        "latency_ms": latency_ms,
        "prompt_tokens": prompt_toks,
        "completion_tokens": completion_toks,
        "usd_cost": cost,
    }

# ──────────────────────────────────────────────────────────────────────────────
# 5. Runner per model
# ──────────────────────────────────────────────────────────────────────────────

async def run_model(model: str) -> None:
    print(f"Running model: {model} …")

    tasks = [call_openai(row, model) for _, row in merged.iterrows()]
    results = await asyncio.gather(*tasks)

    result_df = merged.copy()
    result_df[[
        f"answer_{model}",
        f"latency_ms_{model}",
        f"prompt_tok_{model}",
        f"completion_tok_{model}",
        f"usd_cost_{model}",
    ]] = pd.DataFrame([
        (r["answer"], r["latency_ms"], r["prompt_tokens"], r["completion_tokens"], r["usd_cost"]) for r in results
    ])

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    out_path = os.path.join(
        BASE_DIR,
        "answer_results",
        f"Scenario_QA_with_responses_Simple_{model.replace('.', '_')}_{timestamp}.xlsx",
    )
    result_df.to_excel(out_path, index=False)
    print(f"✅ Saved → {out_path}")

# ──────────────────────────────────────────────────────────────────────────────
# 6. Main – 모델 리스트 실행
# ──────────────────────────────────────────────────────────────────────────────

MODELS = ["gpt-4.1-mini", "gpt-4.1", "gpt-4o", "gpt-4o-mini"]

async def main():
    for model in MODELS:
        await run_model(model)

# 실행 트리거 (스크립트 or 노트북 환경 모두 대응)
try:
    await main()
except RuntimeError:
    nest_asyncio.apply()
    asyncio.get_event_loop().create_task(main())


Running model: gpt-4.1-mini …
✅ Saved → ../data/03_account_login/answer_results/Scenario_QA_with_responses_Simple_gpt-4_1-mini_20250614_064658.xlsx
Running model: gpt-4.1 …
✅ Saved → ../data/03_account_login/answer_results/Scenario_QA_with_responses_Simple_gpt-4_1_20250614_064712.xlsx
Running model: gpt-4o …
✅ Saved → ../data/03_account_login/answer_results/Scenario_QA_with_responses_Simple_gpt-4o_20250614_064720.xlsx
Running model: gpt-4o-mini …
✅ Saved → ../data/03_account_login/answer_results/Scenario_QA_with_responses_Simple_gpt-4o-mini_20250614_064726.xlsx
